# Multiple Page Scrape Function

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [2]:
# one big function to scrape given bay area region by url; returns dataframe

def open_table_scrape(url, max_page):

  # initialize empty lists for restaurant features
  names = []
  ratings = []
  num_reviews = []
  promoted = []
  cities = []
  prices = []
  cuisines = []

  # loop through pages
  pages = np.arange(0, max_page, 100)
  for page in pages:
    
    site = url + str(page)
    
    # grab html from url
    html = urllib.request.urlopen(site).read()
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

    # scrape info
    for resto in soup.find_all('li', class_='result content-section-list-row cf with-times'):

      name = resto.find('span', class_='rest-row-name-text').text
      names.append(name)

      rating = resto.find('div', class_='star-rating-score')['aria-label'] if resto.find('div', class_='star-rating-score') else ''
      ratings.append(rating)

      num_review = resto.find('span', class_='underline-hover').text if resto.find('span', class_='underline-hover') else ''
      num_reviews.append(num_review)

      promote = resto.find('span', class_='promoted-badge').text if resto.find('span', class_='promoted-badge') else ''
      promoted.append(promote.count('Promoted'))

      city = resto.find('span', class_='rest-row-meta--location rest-row-meta-text sfx1388addContent').text
      cities.append(city)

      price = resto.find('i', class_='pricing--the-price').text if resto.find('i', class_="pricing--the-price") else ''
      prices.append(price.count('$'))

      cuisine = resto.find('span', class_='rest-row-meta--cuisine rest-row-meta-text sfx1388addContent').text
      cuisines.append(cuisine)

    # add sleep to prevent server hammering
    sleep(randint(2,7))

  print('names:', len(names))
  print('ratings:', len(ratings))
  print('num_reviews:', len(num_reviews))
  print('promoted:', len(promoted))
  print('cities:', len(cities))
  print('prices:', len(prices))
  print('cuisines:', len(cuisines))

  # create initial dataframe
  restaurants_df = pd.DataFrame({'Name': names,
                                'City': cities,
                                'Rating': ratings,
                                'Review Count': num_reviews,
                                'Promoted': promoted,
                                'Price': prices,
                                'Cuisine': cuisines})
  
  # clean up Rating
  restaurants_df['Rating'] = restaurants_df['Rating'].str.replace(' stars out of 5', '')

  # drop restaurants without ratings/reviews
  restaurants_df['Rating'] = restaurants_df['Rating'].replace('', np.nan)
  restaurants_df = restaurants_df.dropna(subset=['Rating'])
  restaurants_df['Rating'] = restaurants_df['Rating'].astype(float)

  # clean up Review Count
  restaurants_df['Review Count'] = restaurants_df['Review Count'].str.replace(r'[()]', '')
  restaurants_df['Review Count'] = restaurants_df['Review Count'].astype(float)

  # clean up City
  restaurants_df['City'] = restaurants_df['City'].str.replace(', CA', '')

  return restaurants_df

######

# Scraping by region

In [3]:
# create peninsula restaurants dataframe

url = "https://www.opentable.com/san-francisco-bay-area-restaurant-listings?covers=2&currentview=list&datetime=2021-05-27+19%3A00&metroid=4&neighborhoodids%5B%5D=53&neighborhoodids%5B%5D=54&neighborhoodids%5B%5D=120&neighborhoodids%5B%5D=123&neighborhoodids%5B%5D=161&neighborhoodids%5B%5D=533&neighborhoodids%5B%5D=546&neighborhoodids%5B%5D=557&neighborhoodids%5B%5D=634&neighborhoodids%5B%5D=667&neighborhoodids%5B%5D=759&neighborhoodids%5B%5D=908&neighborhoodids%5B%5D=1323&neighborhoodids%5B%5D=1465&neighborhoodids%5B%5D=1519&neighborhoodids%5B%5D=2054&neighborhoodids%5B%5D=3911&neighborhoodids%5B%5D=3929&neighborhoodids%5B%5D=4430&neighborhoodids%5B%5D=5132&neighborhoodids%5B%5D=11858&regionids%5B%5D=8&size=100&sort=Popularity&from="
peninsula_df = open_table_scrape(url, 1001)
peninsula_df.head()

names: 1045
ratings: 1045
num_reviews: 1045
promoted: 1045
cities: 1045
prices: 1045
cuisines: 1045


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,The Village Bakery and Cafe,Woodside,4.7,611.0,1,2,American
1,Osteria,Palo Alto,4.4,880.0,0,2,Italian
2,Cucina Venti,Mountain View,4.6,812.0,0,2,Italian
3,Evvia,Palo Alto,4.8,3602.0,0,3,Mediterranean
4,Taverna,Palo Alto,4.7,671.0,0,3,Greek


In [4]:
# create SF restaurants dataframe

url = "https://www.opentable.com/san-francisco-bay-area-restaurant-listings?covers=2&currentview=list&datetime=2021-06-09+19%3A00&metroid=4&size=100&sort=Popularity&regionids%5B%5D=5&from="
sf_df = open_table_scrape(url, 1801)
sf_df['City'] = 'San Francisco'
sf_df.head()

names: 1805
ratings: 1805
num_reviews: 1805
promoted: 1805
cities: 1805
prices: 1805
cuisines: 1805


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,bota tapas and paella bar,San Francisco,4.2,566.0,0,3,Tapas / Small Plates
2,54 Mint,San Francisco,4.6,2294.0,0,3,Italian
3,Historic John's Grill,San Francisco,4.7,5531.0,0,3,American
4,Doppio Zero - San Francisco,San Francisco,4.4,59.0,0,2,Italian
5,Villon,San Francisco,4.5,628.0,0,3,Californian


In [5]:
# create east bay restaurants dataframe

url = "https://www.opentable.com/san-francisco-bay-area-restaurant-listings?covers=2&currentview=list&datetime=2021-06-09+19%3A00&metroid=4&size=100&sort=Popularity&regionids%5B%5D=6&from="
east_bay_df = open_table_scrape(url, 2401)
east_bay_df

names: 2474
ratings: 2474
num_reviews: 2474
promoted: 2474
cities: 2474
prices: 2474
cuisines: 2474


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,La Finestra Ristorante,Moraga,4.6,201.0,0,2,Italian
1,Barranco,Lafayette,4.4,733.0,0,2,Peruvian
2,American Kitchen,Lafayette,4.6,533.0,0,2,American
3,TELÈFERIC BARCELONA - WALNUT CREEK,Walnut Creek,4.5,2273.0,0,4,Spanish
4,Bridges Restaurant,Danville,4.6,4027.0,0,2,Californian
...,...,...,...,...,...,...,...
2379,Spin a Yarn,Fremont,4.0,457.0,0,3,Continental
2382,Marriott Fremont,Fremont,3.8,41.0,0,3,Contemporary American
2397,The Grill at Poppy Ridge Golf Course,Livermore,4.0,61.0,0,2,American
2403,Arka Restaurant,Fremont,3.3,6.0,0,2,Indian


In [6]:
# create south bay restaurants dataframe

url = "https://www.opentable.com/san-francisco-bay-area-restaurant-listings?covers=2&currentview=list&datetime=2021-06-09+19%3A00&metroid=4&neighborhoodids%5B%5D=51&neighborhoodids%5B%5D=56&neighborhoodids%5B%5D=134&neighborhoodids%5B%5D=157&neighborhoodids%5B%5D=160&neighborhoodids%5B%5D=214&neighborhoodids%5B%5D=726&neighborhoodids%5B%5D=2057&neighborhoodids%5B%5D=4796&neighborhoodids%5B%5D=6143&neighborhoodids%5B%5D=9113&regionids%5B%5D=9&size=100&sort=Popularity&from="
south_bay_df = open_table_scrape(url, 1301)
south_bay_df

names: 1345
ratings: 1345
num_reviews: 1345
promoted: 1345
cities: 1345
prices: 1345
cuisines: 1345


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,Desi,Campbell,3.9,20.0,0,2,Indian
1,Mezcal,San Jose,4.6,556.0,0,2,Regional Mexican
2,The Farmers Union,San Jose,4.4,1264.0,0,2,Contemporary American
3,The Grill on the Alley - San Jose,San Jose,4.6,1548.0,0,4,Steak
4,Scott's Seafood - San Jose,San Jose,4.4,2266.0,0,3,Seafood
...,...,...,...,...,...,...,...
1244,The Good Fork - Morgan Hill,Morgan Hill,4.2,247.0,0,2,Californian
1248,Bubbles,Morgan Hill,4.0,1.0,0,2,Café
1252,88 Keys Cafe,Morgan Hill,3.5,7.0,0,2,American
1293,Cielito Lindo Restaurant,Gilroy,4.2,3.0,0,2,Mexican


In [7]:
# create Marin restaurants dataframe

url = "https://www.opentable.com/san-francisco-bay-area-restaurant-listings?areaid=geohash%3A9qbb58nw&covers=2&currentview=list&datetime=2021-06-09+19%3A00&metroid=4&regionids=7&size=100&sort=Popularity&neighborhoodids%5B%5D=125&neighborhoodids%5B%5D=165&neighborhoodids%5B%5D=169&neighborhoodids%5B%5D=629&neighborhoodids%5B%5D=630&neighborhoodids%5B%5D=958&neighborhoodids%5B%5D=1339&neighborhoodids%5B%5D=1344&neighborhoodids%5B%5D=1345&neighborhoodids%5B%5D=1392&neighborhoodids%5B%5D=3440&originalTerm=Marin&from="
marin_df = open_table_scrape(url, 201)
marin_df

names: 292
ratings: 292
num_reviews: 292
promoted: 292
cities: 292
prices: 292
cuisines: 292


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,Salito's Crab House & Prime Rib,Sausalito,4.5,2227.0,0,2,Seafood
1,Poggio,Sausalito,4.7,4743.0,0,3,Italian
2,The Spinnaker - Sausalito,Sausalito,4.4,4355.0,0,3,Seafood
3,The Trident,Sausalito,4.4,1914.0,0,3,Seafood
4,The Lighthouse Bar & Grill,Mill Valley,4.3,234.0,0,2,American
...,...,...,...,...,...,...,...
257,HopMonk Tavern - Novato,Novato,3.9,220.0,0,2,American
265,Ghiringhelli's Pizzeria Grill & Bar,Novato,3.7,18.0,0,2,Pizzeria
275,Finnegan's Marin,Novato,3.7,26.0,0,2,American
285,Little Goan Indian Cafe,Novato,4.5,1.0,0,2,Indian


In [8]:
# concatenate dataframes

bay_area_df = pd.concat([peninsula_df, sf_df, east_bay_df, south_bay_df, marin_df], ignore_index=True)
bay_area_df

,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,The Village Bakery and Cafe,Woodside,4.7,611.0,1,2,American
1,Osteria,Palo Alto,4.4,880.0,0,2,Italian
2,Cucina Venti,Mountain View,4.6,812.0,0,2,Italian
3,Evvia,Palo Alto,4.8,3602.0,0,3,Mediterranean
4,Taverna,Palo Alto,4.7,671.0,0,3,Greek
...,...,...,...,...,...,...,...
1746,HopMonk Tavern - Novato,Novato,3.9,220.0,0,2,American
1747,Ghiringhelli's Pizzeria Grill & Bar,Novato,3.7,18.0,0,2,Pizzeria
1748,Finnegan's Marin,Novato,3.7,26.0,0,2,American
1749,Little Goan Indian Cafe,Novato,4.5,1.0,0,2,Indian


In [9]:
# Remove restaurants with less than 50 reviews

bay_area_df = bay_area_df[bay_area_df['Review Count'] >= 50].reset_index(drop=True)
bay_area_df

,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,The Village Bakery and Cafe,Woodside,4.7,611.0,1,2,American
1,Osteria,Palo Alto,4.4,880.0,0,2,Italian
2,Cucina Venti,Mountain View,4.6,812.0,0,2,Italian
3,Evvia,Palo Alto,4.8,3602.0,0,3,Mediterranean
4,Taverna,Palo Alto,4.7,671.0,0,3,Greek
...,...,...,...,...,...,...,...
1065,cucina sa,San Anselmo,4.1,401.0,0,3,Italian
1066,Wildfox,Novato,4.0,1004.0,0,2,Comfort Food
1067,Aurora Ristorante Italiano,Novato,4.3,219.0,0,2,Italian
1068,HopMonk Tavern - Novato,Novato,3.9,220.0,0,2,American


In [10]:
# Filter cities

bay_area_df = bay_area_df[bay_area_df['City'] !='Greenbrae']
bay_area_df['City'] = bay_area_df['City'].str.replace('Hillsdale', 'San Mateo')

bay_area_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,City,Rating,Review Count,Promoted,Price,Cuisine
0,The Village Bakery and Cafe,Woodside,4.7,611.0,1,2,American
1,Osteria,Palo Alto,4.4,880.0,0,2,Italian
2,Cucina Venti,Mountain View,4.6,812.0,0,2,Italian
3,Evvia,Palo Alto,4.8,3602.0,0,3,Mediterranean
4,Taverna,Palo Alto,4.7,671.0,0,3,Greek
...,...,...,...,...,...,...,...
1065,cucina sa,San Anselmo,4.1,401.0,0,3,Italian
1066,Wildfox,Novato,4.0,1004.0,0,2,Comfort Food
1067,Aurora Ristorante Italiano,Novato,4.3,219.0,0,2,Italian
1068,HopMonk Tavern - Novato,Novato,3.9,220.0,0,2,American


# Append City Income

In [12]:
# import income csv

city_incomes_df = pd.read_csv('California_incomes.csv')

#clean city incomes df

city_incomes_df = city_incomes_df[['Place', 'Median household income[6]']]
city_incomes_df.columns = ['City', 'Median Household Income (USD)']
city_incomes_df = city_incomes_df.set_index('City')

city_incomes_df['Median Household Income (USD)'] = city_incomes_df['Median Household Income (USD)'].str.replace('$', '')
city_incomes_df['Median Household Income (USD)'] = city_incomes_df['Median Household Income (USD)'].str.replace(',', '')
city_incomes_df['Median Household Income (USD)'] = city_incomes_df['Median Household Income (USD)'].str.replace(r'(\.00)', '')

city_incomes_df['Median Household Income (USD)'] = city_incomes_df['Median Household Income (USD)'].astype(int)

In [14]:
# join city incomes with bay area df

bay_area_join = bay_area_df.join(city_incomes_df, on='City', how='right')
bay_area_join = bay_area_join.reset_index(drop=True)
bay_area_join

,Name,City,Rating,Review Count,Promoted,Price,Cuisine,Median Household Income (USD)
0,Trabocco,Alameda,4.8,2607.0,0.0,3.0,Contemporary Italian,104756
1,The Star on Park,Alameda,4.8,254.0,0.0,2.0,Pizzeria,104756
2,Alley & Vine,Alameda,4.6,131.0,0.0,3.0,Contemporary American,104756
3,Forbidden Island Tiki Lounge,Alameda,4.8,118.0,0.0,2.0,Cocktail Bar,104756
4,The Park Street Tavern,Alameda,4.7,289.0,0.0,3.0,Mediterranean,104756
...,...,...,...,...,...,...,...,...
1065,The Village Bakery and Cafe,Woodside,4.7,611.0,1.0,2.0,American,250001
1066,The Village Pub,Woodside,4.8,2970.0,0.0,3.0,American,250001
1067,Firehouse Bistro,Woodside,4.3,301.0,0.0,3.0,American,250001
1068,Woodside Bakery & Cafe - Permanently Closed,Woodside,3.7,382.0,0.0,2.0,Californian,250001


In [16]:
# export to csv

bay_area_join.to_csv(r"/content/drive/MyDrive/ds_personal_proj/bay_df.csv")